In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import zipfile
import os

# Setup environement

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device in uso:", device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data loading

In [ ]:
zip_path = "/content/drive/MyDrive/AML_MistakeDetection_DATA/features/gopro/segments/1s/video/omnivore.zip"
extract_dir = "/content/omnivore_extracted"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)

extract_dir = "/content/omnivore_extracted/omnivore"

print("Extracted files:", len(os.listdir(extract_dir)))

Extracted files: 384


In [ ]:
import json

with open("/content/drive/MyDrive/AML_MistakeDetection_DATA/annotation_json/complete_step_annotations.json") as f:
    annotations = json.load(f)

In [ ]:
class VideoFeatureDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
def get_labels_for_npz(npz_file, annotations):
    # es: "10_3_360.mp4_1s_1s.npz"
    base = os.path.basename(npz_file)
    activity, attempt = base.split("_")[:2]  # "10", "3"
    recording_id = f"{activity}_{attempt}"

    # carica feature
    data = np.load(npz_file)
    arr = data[list(data.keys())[0]]  # shape (N, 1024)
    N = arr.shape[0]

    labels = np.zeros(N, dtype=np.int64)  # default: normal = no-error = 0

    # trova annotation di questo recording
    info = annotations[recording_id]
    steps = info["steps"]

    # assegnazione label per ogni secondo
    for step in steps:
        has_error = int(step["has_errors"])  # True→1, False→0
        start = step["start_time"]
        end   = step["end_time"]

        if start == -1 or end == -1 or has_error == 0:
            continue

        for sec in range(int(start), int(end) + 1, 1):
            sec_start = sec
            sec_end   = sec + 1

            # check overlap
            if sec_start >= start and sec_end <= end: # i secondi ai bordi avranno sempre il valore di default (norml = no-error = 0)
                labels[sec] = has_error

    return arr, labels

In [ ]:
all_X = []
all_y = []

extract_dir = "/content/omnivore_extracted/omnivore"

for f in sorted(os.listdir(extract_dir)):
    if f.endswith(".npz"):
        X, y = get_labels_for_npz(os.path.join(extract_dir, f), annotations)
        all_X.append(X)
        all_y.append(y)

X = np.concatenate(all_X, axis=0)
y = np.concatenate(all_y, axis=0)

print(X.shape, y.shape)


(340320, 1024) (340320,)


In [ ]:
from sklearn.model_selection import train_test_split

# "Step (S)" data splits (come usato nel paper CaptainCook4D)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    shuffle=True,
)

# implementare anche "Recording (R)" data splits (come fatto nel paper) per vederne differenze

In [ ]:
train_dataset = VideoFeatureDataset(X_train, y_train)
test_dataset  = VideoFeatureDataset(X_test,  y_test)

In [ ]:
batch_size = 512

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# MLP (Version 1)

In [ ]:
import torch
import torch.nn as nn

class MLPCapitainCook(nn.Module):
    def __init__(self, in_features: int, p: float = 0.5) -> None:
        super().__init__()

        self.fc1 = nn.Linear(in_features, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p)       # Dropout layer con probabilità p
        self.fc2 = nn.Linear(256, 1)       # Output logit (senza sigmoid)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)                # Applica Dropout solo in TRAIN
        x = self.fc2(x)                    # Output logit
        return x                           # no Sigmoid qui


In [ ]:
model = MLPCapitainCook(1024).to(device)

In [ ]:
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr)
# count classi
num_error = (y == 1).sum()   # classe “1” = "error"
num_normal = (y == 0).sum()   # classe “0” = "normal"

# pos_weight = quanto pesa la classe “positiva” = classe "1"
#pos_weight_value = num_normal/num_error
pos_weight_value = 1.5
print(pos_weight_value)
pos_weight = torch.tensor([pos_weight_value], device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

epochs = 50

1.5


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

for epoch in range(epochs):

    # -------------------------
    #        TRAIN
    # -------------------------
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs)            # [B, 1]
        outputs = outputs.squeeze(1)       # [B]

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    # -------------------------
    #        EVAL
    # -------------------------
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs).squeeze(1)  # logits

            # same loss as train
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

            # convert logits → probabilities → binary predictions
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).long()

            all_preds.append(preds.cpu())
            all_targets.append(labels.cpu())

    # concat
    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()

    avg_val_loss = total_val_loss / len(test_loader)
    acc = accuracy_score(all_targets, all_preds)
    f1  = f1_score(all_targets, all_preds, zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} "
          f"- Train Loss: {avg_train_loss:.4f} "
          f"- Val Loss: {avg_val_loss:.4f} "
          f"- Acc: {acc:.4f} "
          f"- F1: {f1:.4f}")


Epoch 1/50 - Train Loss: 0.6710 - Val Loss: 0.6475 - Acc: 0.7680 - F1: 0.2537
Epoch 2/50 - Train Loss: 0.6327 - Val Loss: 0.6185 - Acc: 0.7798 - F1: 0.3424
Epoch 3/50 - Train Loss: 0.6049 - Val Loss: 0.5917 - Acc: 0.7903 - F1: 0.4095
Epoch 4/50 - Train Loss: 0.5822 - Val Loss: 0.5715 - Acc: 0.7972 - F1: 0.4712
Epoch 5/50 - Train Loss: 0.5638 - Val Loss: 0.5534 - Acc: 0.8049 - F1: 0.5094
Epoch 6/50 - Train Loss: 0.5471 - Val Loss: 0.5384 - Acc: 0.8108 - F1: 0.5248
Epoch 7/50 - Train Loss: 0.5331 - Val Loss: 0.5260 - Acc: 0.8173 - F1: 0.5480
Epoch 8/50 - Train Loss: 0.5204 - Val Loss: 0.5156 - Acc: 0.8179 - F1: 0.5578
Epoch 9/50 - Train Loss: 0.5097 - Val Loss: 0.5056 - Acc: 0.8226 - F1: 0.5754
Epoch 10/50 - Train Loss: 0.4993 - Val Loss: 0.4971 - Acc: 0.8278 - F1: 0.5817
Epoch 11/50 - Train Loss: 0.4901 - Val Loss: 0.4878 - Acc: 0.8301 - F1: 0.5992
Epoch 12/50 - Train Loss: 0.4821 - Val Loss: 0.4823 - Acc: 0.8329 - F1: 0.6030
Epoch 13/50 - Train Loss: 0.4750 - Val Loss: 0.4744 - Acc: 0.